# Train & Test on Data with Intraclass Variance : ResNet50

In [51]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils import clip_grad_norm_
from torch.optim import Adam,lr_scheduler
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.nn.parallel import DataParallel
import torchvision.models as models
from PIL import Image
import numpy as np
from tqdm import tqdm
import faiss
import os
import pickle
import random
import time
import cv2
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
np.object = np.object_
np.int = np.int_
np.bool = np.bool_

# TRIPLETING

In [52]:
class Triplet:
    def __init__(self, train_folder):
        self.train_folder = train_folder
        self.labels = [label for label in os.listdir(train_folder) if label != '.ipynb_checkpoints']
        self.label_to_path = {label: os.path.join(train_folder, label) for label in self.labels}

    def get_triplet(self):
        anchor_label = random.choice(self.labels)
        anchor_path = random.choice(os.listdir(self.label_to_path[anchor_label]))
        positive_label = anchor_label
        positive_path = random.choice(os.listdir(self.label_to_path[positive_label]))
        negative_label = random.choice([label for label in self.labels if label != anchor_label])
        negative_path = random.choice(os.listdir(self.label_to_path[negative_label]))

        anchor_image = os.path.join(self.label_to_path[anchor_label], anchor_path)
        positive_image = os.path.join(self.label_to_path[positive_label], positive_path)
        negative_image = os.path.join(self.label_to_path[negative_label], negative_path)

        anchor_label_num = self.labels.index(anchor_label)
        positive_label_num = self.labels.index(positive_label)
        negative_label_num = self.labels.index(negative_label)

        return anchor_image, positive_image, negative_image

class TripletDataset(Dataset):
    def __init__(self, train_folder, length, transform=None,):
        self.triplet_generator = Triplet(train_folder)
        self.transform = transform
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        anchor_image, positive_image, negative_image = self.triplet_generator.get_triplet()
        anchor = self._load_image(anchor_image)
        positive = self._load_image(positive_image)
        negative = self._load_image(negative_image)
        return anchor, positive, negative

    def _load_image(self, image_path):
        image = Image.open(image_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image

    def get_triplet_names(self, index):
        anchor_image, positive_image, negative_image = self.triplet_generator.get_triplet()
        return anchor_image, positive_image, negative_image

# TRAIN LOADER

In [53]:
import os
folder_path = "/kaggle/input/animals-insects-reptiles/Animals-Insects-Reptiles/Species-Train"

# Get a list of subfolders in the specified folder
subfolders = [subfolder for subfolder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subfolder))]

total = 0
# Iterate through the subfolders and count the number of files in each
for subfolder in subfolders:
    subfolder_path = os.path.join(folder_path, subfolder)
    num_files = len([filename for filename in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, filename))])
    print(f"Subfolder: {subfolder}, Number of Files: {num_files}")
    total += num_files
print(total)

Subfolder: lynx_canadensis, Number of Files: 130
Subfolder: loxodonta_cyclotis, Number of Files: 130
Subfolder: leptailurus_serval, Number of Files: 130
Subfolder: acinonyx_jubatus, Number of Files: 130
Subfolder: leopardus_wiedii, Number of Files: 130
Subfolder: felis_silvestris, Number of Files: 130
Subfolder: leopardus_pardalis, Number of Files: 130
Subfolder: elephas_maximus, Number of Files: 130
Subfolder: herpailurus_yagouaroundi, Number of Files: 130
Subfolder: felis_lybica, Number of Files: 130
1300


In [54]:
bs = 16
train_folder = "/kaggle/input/animals-insects-reptiles/Animals-Insects-Reptiles/Species-Train"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),           # Convert to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor
])
train_dataset = TripletDataset(train_folder, 1300, transform=transform, )
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

# NETWORK

In [56]:
import torch
import torch.nn as nn
import torchvision.models as models

class TEmbeddingNet(nn.Module):
    def __init__(self, modelt):
        super(TEmbeddingNet, self).__init__()
        self.modelt = modelt
        self.feature_extractor = nn.Sequential(*list(modelt.children())[:-1])

    def forward(self, x):
        features = self.feature_extractor(x)
        return features

    def get_embedding(self, x):
        # Forward pass to get the embedding
        return self.forward(x)

resnet50 = models.resnet50(pretrained=True)
tmodel = TEmbeddingNet(resnet50)

In [57]:
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.enet = embedding_net

    def forward(self, x1, x2=None, x3=None):
        if x2 is None and x3 is None:
            return self.enet.get_embedding(x1)
        return self.enet.get_embedding(x1),self.enet.get_embedding(x2),self.enet.get_embedding(x3)

    def get_embedding(self, x):
        return self.enet.get_embedding(x)

# LOSS, DEVICE, PARAMETERS

In [58]:
class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = torch.norm(anchor - positive, dim=1)
        distance_negative = torch.norm(anchor - negative, dim=1)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()

In [59]:
model = TripletNet(tmodel)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(TripletNet(tmodel))
else:
    model = TripletNet(tmodel)
# Move the model to the selected device (CPU or GPU)
model = model.to(device)

In [60]:
margin = 1
lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=3, gamma=0.5)  # Learning rate scheduler
loss_fn = TripletLoss(margin)
clip_value = 0.5  # You can adjust this value as needed

# FIT

In [61]:
def fit(model, num_epochs, train_loader, bs):
    for epoch in range(n_epochs):
        start = time.time()
        model.train()
        train_loss = 0.0

        for idx, batch in enumerate(train_loader):
            anchor, positive, negative = batch
            anchor = anchor.to(device)
            positive = positive.to(device)
            negative = negative.to(device)

            optimizer.zero_grad()
            anchor_embedding, positive_embedding, negative_embedding = model(anchor, positive, negative)
            anchor_embedding.requires_grad_(True)
            positive_embedding.requires_grad_(True)
            negative_embedding.requires_grad_(True)
            loss = loss_fn(anchor_embedding, positive_embedding, negative_embedding)
            loss.backward()
            clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            train_loss += loss.item()
            if idx%10==0:
                print(loss.item())
        print(f"Epoch {epoch + 1}/{n_epochs}, Train Loss: {train_loss / len(train_loader):.4f}, TIME: {time.time()-start}")
        scheduler.step()


fit(model, n_epochs:=int(input("NO OF EPOCHS : ")), train_loader, bs)


NO OF EPOCHS :  10


0.7187234163284302
0.7841281890869141
0.6014328002929688
0.7398120164871216
0.4712851047515869
1.0727354288101196
0.230413556098938
0.7687028646469116
0.6146390438079834
Epoch 1/10, Train Loss: 0.6748, TIME: 32.89301943778992
0.09229373931884766
0.15297508239746094
0.3165104389190674
0.0
0.5208632946014404
0.1751692295074463
0.11039936542510986
0.3581583499908447
0.5623773336410522
Epoch 2/10, Train Loss: 0.5130, TIME: 32.65100121498108
0.006631970405578613
0.366219699382782
0.31960296630859375
0.9815921783447266
0.530344545841217
0.0
0.8256067633628845
0.9792367219924927
0.2585333585739136
Epoch 3/10, Train Loss: 0.4937, TIME: 32.6024694442749
0.0
0.3833552598953247
0.6225736737251282
0.14969110488891602
0.0
1.5482456684112549
1.0322924852371216
0.08744049072265625
0.8429031372070312
Epoch 4/10, Train Loss: 0.3654, TIME: 32.43656373023987
0.6827393770217896
0.0
0.22038275003433228
0.14798444509506226
0.4803178310394287
0.0
0.23395603895187378
1.2314666509628296
0.5276435613632202
Epoc

# TEST

In [62]:
class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = []
        self.labels = []

        self._load_images()

    def _load_images(self):
        valid_extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
        for class_name in os.listdir(self.folder_path):
            class_folder = os.path.join(self.folder_path, class_name)
            if os.path.isdir(class_folder):
                for filename in os.listdir(class_folder):
                    if filename.lower().endswith(valid_extensions):
                        self.image_paths.append(os.path.join(class_folder, filename))
                        self.labels.append(class_name)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

In [70]:
train_folder = "/kaggle/input/animals-insects-reptiles/Animals-Insects-Reptiles/Species-Train"
train_dataloader = DataLoader(CustomDataset(train_folder,transform=transform))
test_folder = "/kaggle/input/animals-insects-reptiles/Animals-Insects-Reptiles/Species-Test"
test_dataloader = DataLoader(CustomDataset(test_folder,transform=transform))

# EXTRACT EMBEDDINGS

In [71]:
start = time.time()
train_embs = None
train_labels = []
for i in tqdm(train_dataloader):
    I, L = i
    train_labels.append(L)
    emb = tmodel(I.to(device)) # Assuming `model_loaded(I)` returns a PyTorch tensor
    emb = emb.detach()
    if train_embs is None:
        train_embs = emb
    else:
        train_embs = torch.cat((train_embs, emb), dim=0)
print(time.time()-start)

100%|██████████| 1300/1300 [00:15<00:00, 82.21it/s]

15.817509412765503


In [72]:
start = time.time()
test_embs = None
test_labels = []
for i in tqdm(test_dataloader):
    I, L = i
    try:
        emb = tmodel(I.to(device)) # Assuming `model_loaded(I)` returns a PyTorch tensor
        emb = emb.detach()
        if test_embs is None:
            test_embs = emb
        else:
            test_embs = torch.cat((test_embs, emb), dim=0)
        test_labels.append(L)
    except:
        print("ERROR")
print(time.time()-start)

100%|██████████| 200/200 [00:02<00:00, 67.12it/s]

2.9841606616973877


In [73]:
embs_cpu_np = train_embs.cpu().numpy()
embs_cpu_np = embs_cpu_np.reshape(embs_cpu_np.shape[0], -1)
index = faiss.IndexHNSWFlat(embs_cpu_np.shape[1], 32)  # M = 32 for the HNSW index
index.add(embs_cpu_np)

In [74]:
def evaluate_with_faiss(embs, index):
    TOTAL = len(embs)
    CORRECT = 0
    start = time.time()
    
    # Initialize the tqdm progress bar
    with tqdm(total=TOTAL) as pbar:
        for idx, emb in enumerate(embs):
            label = index.search(emb.reshape(1, -1), 1)[1][0][0]
            if train_labels[label][0] == test_labels[idx][0]:
                CORRECT += 1
            pbar.update(1)  # Update the progress bar

    accuracy = (CORRECT / TOTAL) * 100
    elapsed_time = time.time() - start
    return f'Accuracy: {CORRECT}/{TOTAL} = {accuracy:.2f}%, Time: {elapsed_time:.2f} seconds'


In [75]:
embs2_cpu_np = test_embs.cpu().numpy()
embs2_cpu_np = embs2_cpu_np.reshape(embs2_cpu_np.shape[0], -1)

# Accuracy : 56%

In [76]:
print(f'IndexHNSWFlat : {evaluate_with_faiss(embs2_cpu_np,index)}')

100%|██████████| 200/200 [00:00<00:00, 883.66it/s]

IndexHNSWFlat : Accuracy: 113/200 = 56.50%, Time: 0.23 seconds
